<a href="https://colab.research.google.com/github/sevintanerdi/HFD_project/blob/main/notebooks/final_strategy_group2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sevintanerdi/HFD_project.git

Cloning into 'HFD_project'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 400 (delta 144), reused 347 (delta 110), pack-reused 0 (from 0)
Receiving objects: 100% (400/400), 18.90 MiB | 33.83 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [ ]:
!pip install quantstats


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quantstats as qs

In [ ]:
# Settings0
quarters = [
    "2023_Q1", "2023_Q3", "2023_Q4",
    "2024_Q2", "2024_Q4",
    "2025_Q1", "2025_Q2"
]

POINT_VALUE = 100
TCOST = 15

In [ ]:
# function
def mySR(x, scale):
    return np.sqrt(scale) * np.nanmean(x) / np.nanstd(x)

In [ ]:
# summary container
summary_group2_final = pd.DataFrame()

In [ ]:
def calmar_from_pnl(daily_pnl, scale=252):
    cum_pnl = daily_pnl.cumsum()
    drawdown = cum_pnl - cum_pnl.cummax()
    max_dd = drawdown.min()
    if max_dd == 0:
        return np.nan
    return (daily_pnl.mean() * scale) / abs(max_dd)

In [ ]:
pnl_gross_all, pnl_net_all = [], []

for quarter in quarters:

    data = pd.read_parquet(
        f"/content/HFD_project/data_processed/group2_processed/data2_{quarter}_processed.parquet"
    )

    pos_flat = data["pos_flat"].values

    fastEMA = data["XAU"].ewm(span=20).mean()
    slowEMA = data["XAU"].ewm(span=100).mean()

    pos = np.where(
        fastEMA.shift(1).notna() & slowEMA.shift(1).notna(),
        np.where(fastEMA.shift(1) > slowEMA.shift(1), 1, -1),
        np.nan
    )
    pos[pos_flat == 1] = 0

    pnl_gross = np.where(
        np.isnan(pos * data["XAU"].diff()),
        0,
        pos * data["XAU"].diff() * POINT_VALUE
    )

    pnl_net = pnl_gross - np.abs(np.diff(pos, prepend=0)) * TCOST

    pnl_gross_d = pd.Series(pnl_gross, index=data.index.date).groupby(level=0).sum()
    pnl_net_d   = pd.Series(pnl_net,   index=data.index.date).groupby(level=0).sum()
    ntrans_d    = pd.Series(np.abs(np.diff(pos, prepend=0)), index=data.index.date).groupby(level=0).sum()

    pnl_gross_d.index = pd.to_datetime(pnl_gross_d.index)
    pnl_net_d.index   = pd.to_datetime(pnl_net_d.index)

    pnl_gross_all.append(pnl_gross_d)
    pnl_net_all.append(pnl_net_d)

    summary_group2_final = pd.concat(
        [summary_group2_final,
         pd.DataFrame([{
             "quarter": quarter,
             "strategy": "momentum",
             "params": "EMA20-100",
             "gross_SR": mySR(pnl_gross_d, 252),
             "net_SR": mySR(pnl_net_d, 252),
             "gross_PnL": pnl_gross_d.sum(),
             "net_PnL": pnl_net_d.sum(),
             "gross_CR": calmar_from_pnl(pnl_gross_d),
             "net_CR": calmar_from_pnl(pnl_net_d),
             "av_daily_ntrans": ntrans_d.mean(),
             "stat": (mySR(pnl_net_d, 252) - 0.5)
                     * np.maximum(0, np.log(np.abs(pnl_net_d.sum() / 1000)))
         }])],
        ignore_index=True
    )

    plt.figure(figsize=(12, 6))
    plt.plot(pnl_gross_d.cumsum(), label="Gross PnL")
    plt.plot(pnl_net_d.cumsum(), label="Net PnL")
    plt.title(f"XAU EMA(20,100) – {quarter}")
    plt.legend()
    plt.savefig(f"/content/HFD_project/outputs/XAU_EMA20_100_{quarter}.png", dpi=300, bbox_inches="tight")
    plt.close()

pnl_gross_all = pd.concat(pnl_gross_all).sort_index()
pnl_net_all   = pd.concat(pnl_net_all).sort_index()

plt.figure(figsize=(12, 6))
plt.plot(pnl_gross_all.cumsum(), label="Gross PnL")
plt.plot(pnl_net_all.cumsum(), label="Net PnL")
plt.title("Aggregated Cumulative PnL (All Quarters)")
plt.legend()
plt.savefig("/content/HFD_project/outputs/aggregated_cumulative_pnl.png", dpi=300, bbox_inches="tight")
plt.close()

In [ ]:
summary_group2_final

,quarter,strategy,params,gross_SR,net_SR,gross_PnL,net_PnL,gross_CR,net_CR,av_daily_ntrans,stat
0,2023_Q1,momentum,EMA20-100,-0.115801,-1.234443,-958.8,-10528.8,-0.194545,-1.527819,8.179487,-4.083076
1,2023_Q3,momentum,EMA20-100,-3.912499,-5.450587,-21114.7,-30954.7,-2.541408,-2.830500,8.410256,-20.425537
2,2023_Q4,momentum,EMA20-100,3.349136,1.835079,19437.5,10947.5,16.555435,6.775381,7.256410,3.194993
3,2024_Q2,momentum,EMA20-100,1.550795,0.754839,18256.7,9050.7,3.908812,1.657656,7.948718,0.561369
4,2024_Q4,momentum,EMA20-100,1.101248,0.247753,11797.0,2711.0,2.009062,0.427195,7.683544,-0.251570
5,2025_Q1,momentum,EMA20-100,-1.291856,-2.355082,-11664.1,-21759.1,-1.642426,-2.298899,8.740260,-8.793743
6,2025_Q2,momentum,EMA20-100,3.958300,3.506804,77481.0,69285.0,9.795163,8.173719,7.102564,12.743522


In [ ]:
summary_group2_final.to_csv(
    "/content/HFD_project/outputs/summary_group2_XAU_EMA20_100.csv",
    index=False
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 4))
ax.axis("off")

table = ax.table(
    cellText=summary_group2_final.round(3).values,
    colLabels=summary_group2_final.columns,
    loc="center"
)

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

plt.savefig(
    "/content/HFD_project/outputs/summary_stats_group2.png",
    dpi=300,
    bbox_inches="tight"
)

plt.close()
